In [24]:
import sys
import pandas as pd
import numpy as np

# INPUT FILE
in_file = 'price_069500.xlsx'

df1 = pd.read_excel(in_file,dtype='unicode')

# 시계열데이터로 변환
df1.set_index('DATE', inplace=True)
df1.index = pd.to_datetime(df.index)

# 값을 float로 바꾸고, 전일가와 등락률을 계산 (전일가격은 혹시몰라서 가져옴)

In [30]:
# 값을 float로 바꾸고, 전일가와 등락률을 계산 (전일가격은 혹시몰라서 가져옴)
df['PRICE'] = df['PRICE'].astype(float)
df['PREV'] = df['PRICE'].shift(1)
df['RETURN'] = df['PRICE'].pct_change()

In [136]:
def makeTS_03():
	return str(int(datetime.datetime.now().timestamp()))


In [164]:
# test1 시계열 데이터가 다른 종목 간 합쳐보기. 중간 빈데이터는 직전일 가격으로 세팅해보기
import sys
import pandas as pd
import numpy as np
import time, datetime


# SYMBOL
portfolio = ['069500','261220','332940','371450','HERO','SPY']
alloc = [15,15,20,25,15,10]

prc_list = []
for item in portfolio :
    # excel 파일 읽기
    infile = 'price_' + item + '.xlsx'
    df = pd.read_excel(infile,dtype='unicode')
    
    # timeseries로 변환
    df.set_index('DATE', inplace=True)
    df.index = pd.to_datetime(df.index)
    
    # 컬럼명 'PRICE'를 symbol code로 바꾼다
    df.rename(columns={'PRICE':item},inplace=True)
    df[item] = df[item].astype(float)
    prc_list.append(df)

# 하나의 DATAFRAME으로 합친다
prc_df = pd.concat(prc_list,join='outer',axis=1)

# 결측값(N/A)을 처리한다. method='pad'를 넣으면 N/A부분을 직전값으로 대체한다. 
# 상장이 늦게된종목의 경우 앞에서 부터 계속 N/A이므로 대체가 안된다 (나중에 0으로 바꿀지 고민해보자)
prc_df.fillna(method='pad',inplace=True)

# EXCEL에 쓰기 - 시계열에서 시간까지 보여지니 복잡하다. 날짜만 보여주자 => 이거하니까 loc로 접근이 안됨 ㅡ.ㅡ;
# df.index = df.index.date은 concat 후에 변환하자. concat 전에 하면, 늦게 상장한 종목부터 합쳐진다 ㅡ.ㅡ;
#prc_df.index = prc_df.index.date
prc_df.to_excel("concat_result.xlsx")

# 수익률 구하기
rtn_df = prc_df.pct_change()
rtn_df.fillna(0,inplace=True)

# 일별 비중 및 지수계산
wght_df = pd.DataFrame(data=None, columns=rtn_df.columns, index=rtn_df.index)
tot_rtn_df = pd.DataFrame(data=None,columns=['TOT_RTN','INDEX'], index=rtn_df.index)

# 비중 초기화
wght_df.iloc[0] = alloc
wght_df.iloc[0] = wght_df.iloc[0] / 100

# 지수 계산
t1 = time.time()
my_index = 1000.00
for i in range(0, len(wght_df)) :
    #당일 수익계산
    total_return = (wght_df.iloc[i] * rtn_df.iloc[i]).sum() / wght_df.iloc[i].sum()
    tot_rtn_df.iloc[i,0] = total_return
    my_index = my_index * (1+total_return)
    tot_rtn_df.iloc[i,1] = my_index

    #익일 적용할 비중계산 (마지막 ROW는 실행하지 않는다!)
    if i < (len(wght_df) -1):
        wght_df.iloc[i+1] = wght_df.iloc[i] * ( 1+ rtn_df.iloc[i] )
        my_index = my_index+1

t2 = time.time()
elapsed_time = t2 - t1
print(f"처리시간 : {elapsed_time}")



print("finish!!")
tot_rtn_df.to_excel("myindex.xlsx")
    

처리시간 : 1.42918062210083
finish!!


In [3]:
# test2 시계열 데이터가 다른 종목 간 합쳐보기. 중간 빈데이터는 직전일 가격으로 세팅해보기
import sys
import pandas as pd
import numpy as np
import time, datetime


# SYMBOL
portfolio = ['069500','261220','332940','371450','HERO','SPY']
alloc = [15,15,20,25,15,10]

prc_list = []
for item in portfolio :
    # excel 파일 읽기
    infile = 'price_' + item + '.xlsx'
    df = pd.read_excel(infile,dtype='unicode')
    
    # timeseries로 변환
    df.set_index('DATE', inplace=True)
    df.index = pd.to_datetime(df.index)
    
    # 컬럼명 'PRICE'를 symbol code로 바꾼다
    df.rename(columns={'PRICE':item},inplace=True)
    df[item] = df[item].astype(float)
    prc_list.append(df)

# 하나의 DATAFRAME으로 합친다
prc_df = pd.concat(prc_list,join='outer',axis=1)

# 결측값(N/A)을 처리한다. method='pad'를 넣으면 N/A부분을 직전값으로 대체한다. 
# 상장이 늦게된종목의 경우 앞에서 부터 계속 N/A이므로 대체가 안된다 (나중에 0으로 바꿀지 고민해보자)
prc_df.fillna(method='pad',inplace=True)

# EXCEL에 쓰기 
# df.index = df.index.date은 concat 후에 변환하자. concat 전에 하면, 늦게 상장한 종목부터 합쳐진다
# prc_df.index = prc_df.index.date
# prc_df.to_excel("concat_result.xlsx")

# 수익률 구하기
rtn_df = prc_df.pct_change()
rtn_df.fillna(0,inplace=True) # 첫날의 수익률은 0으로 SET

# 일별 비중 및 지수계산
wght_df = pd.DataFrame(data=None, columns=rtn_df.columns, index=rtn_df.index)

# 비중 초기화
wght_df.iloc[0] = alloc
wght_df.iloc[0] = wght_df.iloc[0] / 100

# 지수 계산
t1 = time.time()

# 빠른처리를 위해 dict로 변환 후 iteration 작업
rtn_dict = rtn_df.to_dict('split')
wght_dict = wght_df.to_dict('split')

rtns = rtn_dict['data']
wghts = wght_dict['data']

my_index = 1000
tot_rtn_list = []
my_index_list = []

for i in range(0,len(rtns)) :
    tot_rtn = sum([ x*y for (x,y) in zip(rtns[i], wghts[i]) ]) / sum(wghts[i])
    my_index = my_index * (1+tot_rtn)
    tot_rtn_list.append(tot_rtn)
    my_index_list.append(my_index)
    
    if i < ( len(rtns) -1 ) :
        wghts[i+1] = [ (1+x)*y for (x,y) in zip(rtns[i],wghts[i])]

t2 = time.time()
elapsed_time = t2 - t1
print(f"처리시간 : {elapsed_time}")

index_df = pd.DataFrame({'INDEX':my_index_list, 'TOT_RTN':tot_rtn_list},index=rtn_df.index)

print("finish!!")

index_df.to_excel("myindex_2.xlsx")

처리시간 : 0.018948078155517578
finish!!


In [307]:
sum([ x*y for (x,y) in zip(rtns[1], wghts[1]) ]) / sum(wghts[i])

nan

In [312]:
index_df

,INDEX,TOT_RTN
DATE,,
2018-01-02,1000.000000,0.000000
2018-01-03,1000.371254,0.000371
2018-01-04,1003.994530,0.003622
2018-01-05,1006.262039,0.002258
2018-01-08,1006.467584,0.000204
...,...,...
2021-04-05,1363.912974,0.005781
2021-04-06,1363.494166,-0.000307
2021-04-07,1363.838217,0.000252


In [292]:
# 빠른처리를 위해 dict로 변환 후 iteration 작업
rtn_dict = rtn_df.to_dict('split')
wght_dict = wght_df.to_dict('split')

rtns = rtn_dict['data']
wghts = wght_dict['data']



In [61]:
# test2 시계열 데이터가 다른 종목 간 합쳐보기. 중간 빈데이터는 직전일 가격으로 세팅해보기
import sys
import pandas as pd
import numpy as np

# SYMBOL
portfolio = ['069500','261220','332940','371450','HERO','SPY']
alloc = [15,15,20,25,10,5]

prc_list = []
for item in portfolio :
    # excel 파일 읽기
    infile = 'price_' + item + '.xlsx'
    df = pd.read_excel(infile,dtype='unicode')
    
    # timeseries로 변환
    df.set_index('DATE', inplace=True)
    df.index = pd.to_datetime(df.index)
    
    # 컬럼명 'PRICE'를 symbol code로 바꾼다
    df.rename(columns={'PRICE':item},inplace=True)
    df[item] = df[item].astype(float)
    prc_list.append(df)

In [63]:
prc_list[0].loc['2018-01-02']

069500    30892.0
Name: 2018-01-02 00:00:00, dtype: float64

In [326]:
a = wght_df * rtn_df

In [332]:
a = 1
b = 2
c = a+ b
print(c)

3
